<center> <font size = 5> Predict Heart Disease </font> <center>

Experiments with the Cleveland database have concentrated on simply attempting to distinguish presence (values 1,2,3,4) from absence (value 0). But I am going to attempt a multi-label model first. 

## Table of contents:<br>
- [Preprocessing of data](#preprocessing)
- [Multilabel logistic regression with TensorFlow](#Multilabel_TensorFlow)
- [Multilabel neural network with Keras](#Neural_network_Keras)
- [Binary neural network classification with Keras](#binary_neural_network)

<a id='preprocessing'></a>
### Preprocessing of data

(1) Load data.

In [1]:
import pandas as pd
df=pd.read_csv("Heart disease dataset.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,class
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,1,1,110,264,0,0,132,0,1.2,2,0,7,1
299,68,1,4,144,193,1,0,141,0,3.4,2,2,7,2
300,57,1,4,130,131,0,0,115,1,1.2,2,1,7,3
301,57,0,2,130,236,0,2,174,0,0.0,2,1,3,1


(2) Separate predictor variables and target variables.

In [2]:
X=df.drop(["class"],axis=1)
y=df[["class"]]

(3) One-hot encode the categorical variables among the predictors.

In [3]:
X_categorical=X[["sex","cp","fbs","restecg","exang","slope","ca","thal"]]
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(handle_unknown="ignore")
X_categorical=pd.DataFrame(enc.fit_transform(X_categorical).toarray())
X_categorical

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
299,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
300,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
301,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


(3) Standardize and concatenate.

Standardize the numerical variables among the predictors, then concatenate with the one-hot encoded categorical variables.

In [4]:
X_numerical=X.drop(["sex","cp","fbs","restecg","exang","slope","ca","thal"],axis=1)
from sklearn import preprocessing
names=X_numerical.columns
scaler=preprocessing.StandardScaler()
X_numerical=scaler.fit_transform(X_numerical)
X_numerical=pd.DataFrame(X_numerical, columns=names)
X=pd.concat([X_numerical,X_categorical],axis=1)
X

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,age,trestbps,chol,thalach,oldpeak,0,1,2,3,4,...,15,16,17,18,19,20,21,22,23,24
0,0.948726,0.757525,-0.264900,0.017197,1.087338,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.392002,1.611220,0.760415,-1.821905,0.397182,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,1.392002,-0.665300,-0.342283,-0.902354,1.346147,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-1.932564,-0.096170,0.063974,1.637359,2.122573,0.0,1.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-1.489288,-0.096170,-0.825922,0.980537,0.310912,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,-1.046013,-1.234430,0.334813,-0.770990,0.138373,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
299,1.502821,0.700612,-1.038723,-0.376896,2.036303,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
300,0.283813,-0.096170,-2.238149,-1.515388,0.138373,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
301,0.283813,-0.096170,-0.206864,1.068113,-0.896862,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


<a id='Multilabel_TensorFlow'></a>
### Multi-label logistic regression with TensorFlow

(4) One-hot encode the target variable.

In [5]:
y=pd.DataFrame(enc.fit_transform(y).toarray())

(5) Split the dataset for training and test purposes.

In [6]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.45, random_state=42)

In [7]:
import tensorflow as tf
# numFeatures is the number of features in our input data.
# In the iris dataset, this number is '4'.
numFeatures = trainX.shape[1]

# numLabels is the number of classes our data points can be in.
# In the iris dataset, this number is '3'.
numLabels = trainY.shape[1]


# Placeholders
# 'None' means TensorFlow shouldn't expect a fixed number in that dimension
X = tf.placeholder(tf.float32, [None, numFeatures]) 
yGold = tf.placeholder(tf.float32, [None, numLabels]) 

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/conda/envs/python/lib/python3

In [8]:
#Randomly sample from a normal distribution with standard deviation .01

weights = tf.Variable(tf.random_normal([numFeatures,numLabels],
                                       mean=0,
                                       stddev=0.001,
                                       name="weights"))

bias = tf.Variable(tf.random_normal([1,numLabels],
                                    mean=0,
                                    stddev=0.001,
                                    name="bias"))

In [9]:
# Three-component breakdown of the Logistic Regression equation.
# Note that these feed into each other.
apply_weights_OP = tf.matmul(X, weights, name="apply_weights")
add_bias_OP = tf.add(apply_weights_OP, bias, name="add_bias") 
activation_OP = tf.nn.sigmoid(add_bias_OP, name="activation")

In [10]:
# Number of Epochs in our training
numEpochs = 50000

# Defining our learning rate iterations (decay)
learningRate = tf.train.exponential_decay(learning_rate=0.0001,
                                          global_step= 1,
                                          decay_steps=trainX.shape[0],
                                          decay_rate= 0.9,
                                          staircase=True)

In [11]:
#Defining our cost function - Squared Mean Error
cost_OP = tf.nn.l2_loss(activation_OP-yGold, name="squared_error_cost")

#Defining our Gradient Descent
training_OP = tf.train.GradientDescentOptimizer(learningRate).minimize(cost_OP)

In [12]:
# Create a tensorflow session
sess = tf.Session()

# Initialize our weights and biases variables.
init_OP = tf.global_variables_initializer()

# Initialize all tensorflow variables
sess.run(init_OP)

In [13]:
# argmax(activation_OP, 1) returns the label with the most probability
# argmax(yGold, 1) is the correct label
correct_predictions_OP = tf.equal(tf.argmax(activation_OP,1),tf.argmax(yGold,1))

# If every false prediction is 0 and every true prediction is 1, the average returns us the accuracy
accuracy_OP = tf.reduce_mean(tf.cast(correct_predictions_OP, "float"))

# Summary op for regression output
activation_summary_OP = tf.summary.histogram("output", activation_OP)

# Summary op for accuracy
accuracy_summary_OP = tf.summary.scalar("accuracy", accuracy_OP)

# Summary op for cost
cost_summary_OP = tf.summary.scalar("cost", cost_OP)

# Summary ops to check how variables (W, b) are updating after each iteration
weightSummary = tf.summary.histogram("weights", weights.eval(session=sess))
biasSummary = tf.summary.histogram("biases", bias.eval(session=sess))

# Merge all summaries
merged = tf.summary.merge([activation_summary_OP, accuracy_summary_OP, cost_summary_OP, weightSummary, biasSummary])

# Summary writer
writer = tf.summary.FileWriter("summary_logs", sess.graph)

In [14]:
# Initialize reporting variables
cost = 0
diff = 1
epoch_values = []
accuracy_values = []
cost_values = []

# Training epochs
for i in range(numEpochs):
    if i > 1 and diff < .0001:
        print("change in cost %g; convergence."%diff)
        break
    else:
        # Run training step
        step = sess.run(training_OP, feed_dict={X: trainX, yGold: trainY})
        # Report occasional stats
        if i % 1000 == 0:
            # Add epoch to epoch_values
            epoch_values.append(i)
            # Generate accuracy stats on test data
            train_accuracy, newCost = sess.run([accuracy_OP, cost_OP], feed_dict={X: trainX, yGold: trainY})
            # Add accuracy to live graphing variable
            accuracy_values.append(train_accuracy)
            # Add cost to live graphing variable
            cost_values.append(newCost)
            # Re-assign values for variables
            diff = abs(newCost - cost)
            cost = newCost

            #generate print statements
            print("step %d, training accuracy %g, cost %g, change in cost %g"%(i, train_accuracy, newCost, diff))


# How well do we perform on held-out test data?
print("final accuracy on test set: %s" %str(sess.run(accuracy_OP, 
                                                     feed_dict={X: testX, 
                                                                yGold: testY})))

step 0, training accuracy 0.548193, cost 103.163, change in cost 103.163
step 1000, training accuracy 0.596386, cost 41.7096, change in cost 61.4537
step 2000, training accuracy 0.626506, cost 38.7719, change in cost 2.9377
step 3000, training accuracy 0.686747, cost 37.2892, change in cost 1.48273
step 4000, training accuracy 0.686747, cost 36.318, change in cost 0.97123
step 5000, training accuracy 0.686747, cost 35.607, change in cost 0.710991
step 6000, training accuracy 0.674699, cost 35.0531, change in cost 0.553917
step 7000, training accuracy 0.680723, cost 34.6041, change in cost 0.44899
step 8000, training accuracy 0.686747, cost 34.2299, change in cost 0.37421
step 9000, training accuracy 0.698795, cost 33.9113, change in cost 0.318581
step 10000, training accuracy 0.704819, cost 33.6354, change in cost 0.275917
step 11000, training accuracy 0.710843, cost 33.393, change in cost 0.242386
step 12000, training accuracy 0.710843, cost 33.1775, change in cost 0.215515
step 13000

**Before proceeding further, restart the kernel and run (1) to (3). Then, run the following code blocks.**

<a id='Neural_network_Keras'></a>
### Multilabel neural network model with Keras

In [5]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.4, random_state=42)

In [6]:
length=trainX.shape[1]
from keras.utils import to_categorical
trainY=to_categorical(trainY)
testY=to_categorical(testY)
num_classes=testY.shape[1]

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [7]:
# Load the libraries
import keras
from keras.models import Sequential
from keras.layers import Dense

# define classification model

def classification_model():
    # create model
    model = Sequential()
    model.add(Dense(length, activation='relu', input_shape=(length,)))
    model.add(Dense(80,activation='relu'))
    model.add(Dense(80,activation="relu"))
    model.add(Dense(80,activation="sigmoid"))
    model.add(Dense(num_classes, activation='softmax'))
    
    
    # compile model
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
model=classification_model()
model.fit(trainX,trainY,validation_data=(testX, testY),epochs=15, verbose=2)
model.evaluate(testX,testY,verbose=0)

Train on 181 samples, validate on 122 samples
Epoch 1/15
 - 1s - loss: 1.4752 - acc: 0.5525 - val_loss: 1.4373 - val_acc: 0.5246
Epoch 2/15
 - 0s - loss: 1.3369 - acc: 0.5525 - val_loss: 1.3432 - val_acc: 0.5246
Epoch 3/15
 - 1s - loss: 1.2633 - acc: 0.5525 - val_loss: 1.2846 - val_acc: 0.5246
Epoch 4/15
 - 0s - loss: 1.2152 - acc: 0.5525 - val_loss: 1.2372 - val_acc: 0.5246
Epoch 5/15
 - 0s - loss: 1.1733 - acc: 0.5525 - val_loss: 1.1933 - val_acc: 0.5246
Epoch 6/15
 - 0s - loss: 1.1170 - acc: 0.5525 - val_loss: 1.1393 - val_acc: 0.5246
Epoch 7/15
 - 0s - loss: 1.0535 - acc: 0.5635 - val_loss: 1.0770 - val_acc: 0.5328
Epoch 8/15
 - 0s - loss: 0.9910 - acc: 0.6022 - val_loss: 1.0252 - val_acc: 0.5492
Epoch 9/15
 - 0s - loss: 0.9336 - acc: 0.6188 - val_loss: 0.9917 - val_acc: 0.5738
Epoch 10/15
 - 0s - loss: 0.8919 - acc: 0.6243 - val_loss: 0.9736 - val_acc: 0.5738
Epoch 11/15
 - 0s - loss: 0.8607 - acc: 0.6464 - val_loss: 0.9590 - val_acc: 0.5738
Epoch 12/15
 - 0s - loss: 0.8318 - acc:

[0.9416586248601069, 0.5819672180003808]

58.2%. **That was a bold try.** That seems to be the best we could get on the cleveland dataset. The dataset is stubbornly difficult to model. No wonder why "experiments with the Cleveland database have concentrated on simply attempting to distinguish presence (values 1,2,3,4) from absence (value 0)".

It seems that we need to turn the multi-class problem into a binary classification problem: no heart disease vs. heart disease. This time, instead of connecting to IBM's database in order to write SQL query, I am just going to write a few lines of codes here.

<a id='binary_neural_network'></a>
### Binary neural network model with Keras

**Let's re-start the kernel and re-load the data to make sure that work from this point on won't be affected by previous results.**

In [1]:
import pandas as pd
df=pd.read_csv("Heart disease dataset.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,class
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,1,1,110,264,0,0,132,0,1.2,2,0,7,1
299,68,1,4,144,193,1,0,141,0,3.4,2,2,7,2
300,57,1,4,130,131,0,0,115,1,1.2,2,1,7,3
301,57,0,2,130,236,0,2,174,0,0.0,2,1,3,1


In [2]:
y=df[["class"]].replace([1,2,3,4],1)
y

,class
0,0
1,1
2,1
3,0
4,0
...,...
298,1
299,1
300,1
301,1


We have turned the multi-class problem into a binary problem.

As before, I am going to one-hot encode the categorical predictor variables and standardize the numerical ones.

In [3]:
X=df.drop(["class"],axis=1)
X_categorical=X[["sex","cp","fbs","restecg","exang","slope","ca","thal"]]
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(handle_unknown="ignore")
X_categorical=pd.DataFrame(enc.fit_transform(X_categorical).toarray())
X_categorical

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
299,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
300,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
301,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [4]:
X_numerical=X.drop(["sex","cp","fbs","restecg","exang","slope","ca","thal"],axis=1)
from sklearn import preprocessing
names=X_numerical.columns
scaler=preprocessing.StandardScaler()
X_numerical=scaler.fit_transform(X_numerical)
X_numerical=pd.DataFrame(X_numerical, columns=names)
X=pd.concat([X_numerical,X_categorical],axis=1)
X

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,age,trestbps,chol,thalach,oldpeak,0,1,2,3,4,...,15,16,17,18,19,20,21,22,23,24
0,0.948726,0.757525,-0.264900,0.017197,1.087338,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.392002,1.611220,0.760415,-1.821905,0.397182,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,1.392002,-0.665300,-0.342283,-0.902354,1.346147,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-1.932564,-0.096170,0.063974,1.637359,2.122573,0.0,1.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-1.489288,-0.096170,-0.825922,0.980537,0.310912,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,-1.046013,-1.234430,0.334813,-0.770990,0.138373,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
299,1.502821,0.700612,-1.038723,-0.376896,2.036303,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
300,0.283813,-0.096170,-2.238149,-1.515388,0.138373,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
301,0.283813,-0.096170,-0.206864,1.068113,-0.896862,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


Split the data for training and test purposes.

In [5]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.11, random_state=42)

Build the neural network model.

In [6]:
length=trainX.shape[1]
from keras.utils import to_categorical
trainY=to_categorical(trainY)
testY=to_categorical(testY)
num_classes=testY.shape[1]

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [7]:
# Load the libraries
import keras
from keras.models import Sequential
from keras.layers import Dense

# define classification model

def classification_model():
    # create model
    model = Sequential()
    model.add(Dense(length, activation='relu', input_shape=(length,)))
    model.add(Dense(60,activation='relu'))
    model.add(Dense(60,activation="relu"))
    model.add(Dense(60,activation="tanh"))
    model.add(Dense(num_classes, activation='softmax'))
    
    
    # compile model
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [10]:
model=classification_model()
model.fit(trainX,trainY,validation_data=(testX, testY),epochs=20, verbose=2)
model.evaluate(testX,testY,verbose=0)

Train on 269 samples, validate on 34 samples
Epoch 1/20
 - 4s - loss: 0.6687 - acc: 0.5874 - val_loss: 0.6191 - val_acc: 0.7353
Epoch 2/20
 - 0s - loss: 0.5941 - acc: 0.7323 - val_loss: 0.5274 - val_acc: 0.8235
Epoch 3/20
 - 0s - loss: 0.5081 - acc: 0.7844 - val_loss: 0.4552 - val_acc: 0.7941
Epoch 4/20
 - 0s - loss: 0.4515 - acc: 0.7918 - val_loss: 0.3478 - val_acc: 0.8529
Epoch 5/20
 - 0s - loss: 0.3993 - acc: 0.8141 - val_loss: 0.3198 - val_acc: 0.8824
Epoch 6/20
 - 0s - loss: 0.3709 - acc: 0.8327 - val_loss: 0.2650 - val_acc: 0.9118
Epoch 7/20
 - 0s - loss: 0.3421 - acc: 0.8550 - val_loss: 0.2578 - val_acc: 0.9118
Epoch 8/20
 - 0s - loss: 0.3182 - acc: 0.8699 - val_loss: 0.2139 - val_acc: 0.9118
Epoch 9/20
 - 0s - loss: 0.2956 - acc: 0.8810 - val_loss: 0.2160 - val_acc: 0.9118
Epoch 10/20
 - 0s - loss: 0.2781 - acc: 0.8848 - val_loss: 0.1990 - val_acc: 0.9118
Epoch 11/20
 - 0s - loss: 0.2566 - acc: 0.8959 - val_loss: 0.2055 - val_acc: 0.9118
Epoch 12/20
 - 0s - loss: 0.2427 - acc: 

[0.2097664194948533, 0.9411764705882353]

In [11]:
from sklearn.metrics import roc_auc_score
y_pred=model.predict(testX)
print("The AUC on the test data is: ",roc_auc_score(testY,y_pred))

The AUC on the test data is:  0.9719298245614035


Accuracy of 94.12% and AUC of 0.972 on **test data**. Not bad at all. Indeed, the model is excellent.